# Import requirements

In [1]:
from __future__ import division
import pandas as pd
import numpy as np
from collections import Counter
import time
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline

# Import data

In [6]:
C = ['Patient','Visit','DateOfVisit','Age','Gender','ART','VL','iVL','pVL',
     'CD4','iCD4','nCD4','CD8','iCD8','nCD8','TMHDS','VisitDate','GDS','Prot', 
     'AAPos','Coverage','A','R','N','D','C','Q','E','G','H','I','L','K','M',
     'F','P','S','T','W','Y','V']

In [7]:
DF = pd.read_csv('/Users/greg/Desktop/FullNeuroIllumina/MergedData/NGS_GDS_abundance.csv')
print DF.shape
DF = DF[C]
print DF.shape
DF.head()

(5732, 57)
(5732, 41)


,Patient,Visit,DateOfVisit,Age,Gender,ART,VL,iVL,pVL,CD4,...,L,K,M,F,P,S,T,W,Y,V
0,A0001,R09,2014-11-10,59,Male,on,20,987,987,797,...,0.006934,0.001695,0.565052,0.000026,0,0.000848,0.000128,0.416637,0,0.000205
1,A0010,R08,2014-11-12,59,Male,on,20,50,470,1167,...,0.005848,0.005848,0.584795,0.000000,0,0.000000,0.000000,0.403509,0,0.000000
2,A0013,R09,2014-11-10,68,Male,on,20,144,39373,771,...,0.001661,0.004983,0.282392,0.000000,0,0.001661,0.000000,0.209302,0,0.000000
3,A0019,R12,2015-02-02,46,Female,on,20,99,164020,1130,...,0.008081,0.003367,0.526599,0.000000,0,0.001347,0.000673,0.452525,0,0.000000
4,A0026,R09,2015-04-30,55,Male,on,67859,100000,100000,137,...,0.008982,0.000000,0.589820,0.000000,0,0.002994,0.000000,0.389222,0,0.000000


# Functions

In [ ]:
sample_col = ['Patient','Visit']
demographics_col = ['Age', 'Gender', 'ART']
chemistry_col = ['VL','iVL','pVL','CD4','iCD4','nCD4','CD8','iCD8',u'nCD8']
neuro_col =['TMHDS','GDS']
sequence_col = ['Prot','AAPos','Coverage']
aa_col = ['A','R','N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','Y','V']

def slice_position(df, pos):
    m = df['AAPos'] == pos
    sliced_df = df[m]
    return sliced_df

def chooseDataCols(pos_df):
    cols = []
    cols.extend(sample_col)
    cols.extend(neuro_col)
    cols.extend(aa_col)
    data = pos_df[cols].copy()
    data['intercept'] = 1.0
    return data

def filterX(X, threshold):
    t = X.max()>threshold
    #t = X.mean()>threshold
    filtered_cols = t[t==True].index.tolist()
    X = X[filtered_cols]
    return X

def getXandY(data, threshold):   
    train_cols = []
    train_cols.extend(aa_col)
    train_cols.extend(['intercept'])
    y = (data['GDS'] >= 0.5).astype(int)
    X = data[train_cols]
    X = filterX(X, threshold)
    return X, y